## imports

In [ ]:
import os
from geopandas import read_file
from geojson import Feature, FeatureCollection, dump

data_partition = 'd:\\data\\resow'

hexgrid_dir = os.path.join(data_partition, 'hexgrid')
base_file_path = os.path.join(hexgrid_dir, 'hexgrid_base.geojson')
out_file_path = os.path.join(hexgrid_dir, 'hexgrid2.geojson')

base_grid = read_file(base_file_path)

crs = {
        "type": "name",
        "properties": {"name": base_grid.crs['init']}
      }

def WKT(hexgrid_df):
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    print(f'POLYGON(({west} {north}, {west} {south} ,' + \
           f'{east} {south}, {east} {north}, {west} {north}))')

## run the code

In [ ]:
features = []
for index, row in base_grid.iterrows():
    features.append(Feature(geometry=row['geometry'],
                            properties={'id': index}))

feature_collection = FeatureCollection(features, crs=crs)

with open(out_file_path, 'w') as f:
   dump(feature_collection, f)

In [ ]:
for cell in [23, 63, 88, 128, 204, 258]:
    
    hexcell_file_path = os.path.join(hexgrid_dir, f'hexgrid_{str(cell)}.geojson')

    features = []
    for index, row in base_grid.iterrows():
        
        if index == cell:
            features.append(Feature(geometry=row['geometry'],
                            properties={'id': index,
                                        'west': row['left'],
                                        'east': row['right'],
                                        'south': row['bottom'],
                                        'north': row['top']}))
            
    feature_collection = FeatureCollection(features, crs=crs)

    with open(hexcell_file_path, 'w') as f:
       dump(feature_collection, f)
    
    WKT(read_file(hexcell_file_path))
    print()